In [10]:
#Numpy for array related functions.
import numpy as np
#Dlib for deep learning based Modules and face landmark detection.
import dlib
#Import OpenCV Library for basic image processing functions.
import cv2
#face_utils for basic operations of conversion
import imutils
from imutils import face_utils
#scipy to solve scientific and mathematical problems
from scipy.spatial import distance as dist


def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])   #vertical distance
    C = dist.euclidean(eye[0], eye[3])   #horizontal distance

    ear = (A + B) / (2.0 * C)
    return ear

#Grab the indexes of the facial landmarks for the left and
# right eye,respectively
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
( rStart , rEnd ) =face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

cap = cv2.VideoCapture(0)

#Initializing the face detector and landmark detector
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

while True:
    ret, frame = cap.read()
    if ret == False:
        print('Failed to capture frame from camera. Check camera index in cv2.VideoCapture(0) \n')
        break
        cv2.imshow(frame)

    frame = imutils.resize(frame, width=650)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = detector(gray)

    for rect in rects:
        
        landmarks = predictor(gray, rect)
        landmarks = face_utils.shape_to_np(landmarks)
        
        shape = predictor(gray, rect)    # determine the facial landmarks for face region
        shape = face_utils.shape_to_np(shape) #converting to numpy array

        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]

        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)

        ear = (leftEAR + rightEAR) / 2.0

        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

       

        cv2.putText(frame, "PLEASE NOTE YOUR APPROX. OPEN EYE EAR VALUE: {:.2f}".format(ear), (0, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0,7), 2)
        
    
    cv2.imshow("Frame", frame)
    
    key = cv2.waitKey(1) & 0xFF

    if key == ord("q"):
        break

cv2.destroyAllWindows()
cap.release()